In [ ]:
#Importing all necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import time

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from openpyxl import load_workbook

In [ ]:
#Reading the excel-document into a df
df = pd.read_excel('dataset_name.xlsx')

#dropping unwanted columns from the df
df2=df.drop(columns = ["Column 1", "Column 2"])


In [ ]:
#Splitting df into two based on ID
df2["ID"] = df2["ID"].astype(str)  #Making sure its a string
df_train = df2[df2["ID"].isin(["ID 1", "ID 2", "ID 3"])]
df_unseen = df2[df2["ID"] == "ID 4"]

In [ ]:
#defining which columns to keep av model input
X = df_train.drop(columns = ['Column 1', 'Column 2', 'Column 3'])

#defining which columns are the model outputs
y = df_train['Column 4']

#defining inputs and outputs for unseen dataset
X_unseen = df_unseen.drop(columns = ['Column 1', 'Column 2', 'Column 3')
y_unseen = df_unseen['Column 4']

#Setting up k-fold cross validation (with 5 splits, shuffling, and a fixed random seed for reproducibility)
k_fold = KFold(n_splits=5, random_state=66, shuffle=True)

In [ ]:
#Defining a pipeline that first scales the features, then applying linear regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),     
    ('mlr', LinearRegression())    
])

#Initializing empty lists to store performance metrics for each fold
mae_scores_train, mse_scores_train, r2_scores_train = [], [], []
mae_scores_test, mse_scores_test, r2_scores_test = [], [], []

#Performing k-fold cross-validation
for train_index, test_index in k_fold.split(X, y):
    # Split the data into training and test sets for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #Skipping the fold if the test set contains only one unique value (to avoid undefined R²)
    if len(np.unique(y_test)) == 1:
        print("Skipping this fold due to only one class in test set.")
        continue

    #Training the pipeline (scaling  and regression) on the training data
    pipeline.fit(X_train, y_train)

    #Making predictions for both training and test data
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    #Computing regression metrics for training data
    mae_train = mean_absolute_error(y_train, pred_train)
    mse_train = mean_squared_error(y_train, pred_train)
    r2_train = r2_score(y_train, pred_train)

    #Computing regression metrics for test data
    mae_test = mean_absolute_error(y_test, pred_test)
    mse_test = mean_squared_error(y_test, pred_test)
    r2_test = r2_score(y_test, pred_test)

    #Appending metrics to their lists for later averaging
    mae_scores_train.append(mae_train)
    mse_scores_train.append(mse_train)
    r2_scores_train.append(r2_train)

    mae_scores_test.append(mae_test)
    mse_scores_test.append(mse_test)
    r2_scores_test.append(r2_test)

    #Printing performance metrics for the current fold
    print(f"Train -> MAE: {mae_train:.6f}, MSE: {mse_train:.6f}, R²: {r2_train:.6f}")
    print(f"Test  -> MAE: {mae_test:.6f}, MSE: {mse_test:.6f}, R²: {r2_test:.6f}\n")

#Printing the average metrics across all folds
print("\nAverage results for all folds:")
print(f"Train Mean MAE: {np.mean(mae_scores_train):.6f}, Test Mean MAE: {np.mean(mae_scores_test):.6f}")
print(f"Train Mean MSE: {np.mean(mse_scores_train):.6f}, Test Mean MSE: {np.mean(mse_scores_test):.6f}")
print(f"Train Mean R²: {np.mean(r2_scores_train):.6f}, Test Mean R²: {np.mean(r2_scores_test):.6f}")


In [ ]:
#Defining a machine learning pipeline that includes:
#1. Scaling the input features
#2. Expanding the features to include polynomial terms (degree 2)
#3. Applying a regularized linear regression model
pipeline = Pipeline([
    ('scaler', StandardScaler()),                     #Standardizing the features
    ('poly', PolynomialFeatures(degree=2)),           #Adding polynomial terms
    ('reg', Ridge())                                  #Applying Ridge regression
])

#Defining a grid of hyperparameters to search over
parameters_grid = {
    'poly__degree': [1,2,3],                          #Degree of polynomial features 
    'reg__alpha': [5, 10, 15, 20, 25],                #Regularization strength for Ridge (higher = more regularization)
    'reg__fit_intercept': [True],                     #Whether to fit the intercept (True = yes)
    'reg__solver': ['auto', 'svd', 'cholesky', 'lsqr']#Solvers supported by Ridge regression
}

#Setting up a GridSearchCV to perform a search over the parameter grid using cross-validation
CV_MLR = GridSearchCV(
    estimator=pipeline,               #Useing the pipeline as the estimator
    param_grid=parameters_grid,       #Dictionary of hyperparameters to test
    cv=k_fold,                        #Using defined k-fold as cross validation
    scoring='r2',                     #Setting R² as the scoring metric
    n_jobs=-1                         #Useing all available CPU cores for parallel processing
)

#Fitting the grid search on the entire dataset
CV_MLR.fit(X, y)

#Output the best combination of parameters found during grid search
print('Best parameters: ', CV_MLR.best_params_)

In [ ]:
#Getting the best model and parameters from the GridSearchCV
best_pipeline = CV_MLR.best_estimator_  
best_params = CV_MLR.best_params_      

#Defining lists to store R² and MSE scores for each fold
r2_train_scores = []
r2_test_scores = []
mse_train_scores = []
mse_test_scores = []

#Performing evaluation using the same KFold splits
for train_idx, test_idx in k_fold.split(X, y):
    # Split the data into training and test sets for the current fold
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    #Skipping fold if test target contains only one unique value to make sure calculation of R² will work
    if len(np.unique(y_test)) == 1:
        print("Skipping fold due to constant test targets.")
        continue

    #Re-training the best model on this fold's training data
    best_pipeline.fit(X_train, y_train)

    #Generating predictions for both training and test sets
    y_train_pred = best_pipeline.predict(X_train)
    y_test_pred = best_pipeline.predict(X_test)

    #Calculating and store R² and Mean Squared Error for both train and test
    r2_train_scores.append(r2_score(y_train, y_train_pred))
    r2_test_scores.append(r2_score(y_test, y_test_pred))
    mse_train_scores.append(mean_squared_error(y_train, y_train_pred))
    mse_test_scores.append(mean_squared_error(y_test, y_test_pred))

#Printing the average performance across all valid folds
print("\n[FINAL MLR - Refit on each KFold split]")
print(f"Train R²: {np.mean(r2_train_scores):.4f}, MSE: {np.mean(mse_train_scores):.4f}")
print(f"Test  R²: {np.mean(r2_test_scores):.4f}, MSE: {np.mean(mse_test_scores):.4f}")


In [ ]:
#Refitting best model on all training data before evaluating on unseen
best_pipeline.fit(X, y)

#calculating predictions on unseen data
y_unseen_pred = best_pipeline.predict(X_unseen)

mse_unseen = mean_squared_error(y_unseen, y_unseen_pred)
r2_unseen = r2_score(y_unseen, y_unseen_pred)

print(f'Unseen Mean Squared Error: {mse_unseen:.4f}')
print(f'Unseen R-squared: {r2_unseen:.4f}')

In [ ]:
avg_mse_train = np.mean(mse_train_scores)
avg_r2_train = np.mean(r2_train_scores)
avg_mse_test = np.mean(mse_test_scores)
avg_r2_test = np.mean(r2_test_scores)

In [ ]:
#Defining a function to log model evaluation metrics and hyperparameters to an Excel-file of chosen name
def log_results(model_name, params, mse_train, r2_train, mse_test, r2_test, mse_unseen, r2_unseen, filename="Excel_name.xlsx"):
    """ Logs model results in separate sheets within the same Excel file, ensuring appending works correctly. """

    #Creating DataFrame for this model run
    result = pd.DataFrame([{
        **params,  # Store hyperparameters
        "MSE_Train": mse_train,
        "R2_Train": r2_train,
        "MSE_Test": mse_test,
        "R2_Test": r2_test,
        "MSE_Unseen": mse_unseen,
        "R2_Unseen": r2_unseen
    }])

    #Introducing a short delay to avoid file conflicts if running in multiple notebooks
    time.sleep(1)

    #Checking if the file exists
    file_exists = os.path.exists(filename)

    if not file_exists:
        #If file doesn't exist, creating a new one
        with pd.ExcelWriter(filename, engine="openpyxl", mode="w") as writer:
            result.to_excel(writer, sheet_name=model_name, index=False)
    else:
        #If file exists, loading it properly before appending
        try:
            with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
                #Reading existing sheet 
                try:
                    existing_df = pd.read_excel(filename, sheet_name=model_name, engine="openpyxl")
                    df_combined = pd.concat([existing_df, result], ignore_index=True)
                except (FileNotFoundError, ValueError):
                    df_combined = result  # If sheet does not exist, creating it

                #Saveing results, ensuring correct appending
                df_combined.to_excel(writer, sheet_name=model_name, index=False)

        except PermissionError:
            #Printing error to warn user
            print(f"Error: Close the Excel file ({filename}) before running the script again.")
    #Printing a confirmation to ensure user results are logged
    print(f"Logged results for {model_name}: Train MSE={mse_train:.4f}, Test MSE={mse_test:.4f}, Unseen MSE={mse_unseen:.4f}")


In [ ]:
#Using the function to log results from MLR model
log_results(
    model_name="MultipleLinearRegression",
    params=best_params,
    mse_train=avg_mse_train,
    r2_train=avg_r2_train,
    mse_test=avg_mse_test,
    r2_test=avg_r2_test,
    mse_unseen=mse_unseen,
    r2_unseen=r2_unseen
)